# Logistic Regression (and some Linear Regression)
This is a somewhat longer introduction to Logistic Regression and how it is connected to Deep Learning.

This notebook is also available as a [blog post](https://towardsdatascience.com/a-logistic-regression-from-scratch-3824468b1f88) and on [Github](https://github.com/dennisbakhuis/Tutorials/tree/master/Logistic_Regression).

If you are new to Python, here is a course I wrote to get started with Python:\
[https://python-10-minutes-a-day.rocks](https://python-10-minutes-a-day.rocks)

### A short overview of the topics we will be discussing:
1. Link between neural networks and logistic regression
2. One step back: linear regression
3. From linear to (binary) logistic regression
4. Round up

# 1. Link between neural network and logistic regression
When we hear or read about deep learning we generally mean the sub-field of machine learning using artificial neural networks (ANN). These computing systems are quite successful in solving complex problems in various fields, examples are, image recognition, language modelling, and speech recognition. While the name ANN implies that they are related to the inner workings of our brain, the truth is that they mainly share some terminology. An ANN generally consists of multiple interconnected layers, which on itself are build using neurons (also called nodes). An example is shown in figure 1.

![Neural Network](https://github.com/dennisbakhuis/Tutorials/raw/master/Neural_Network/assets/NeuralNetwork.png)
*Figure 1: Example of a typical neural network, borrowed from my follow up article on neural networks.*

In this example, we have one input layer, consisting of four individual inputs nodes. This input layer is ‘fully connected’ to the first hidden layer, i.e. Fully connected means that each input is connected to each node. The first hidden layer is again fully connected to another ‘hidden’ layer. The term hidden indicates that we are not directly interact with these layers and these are kind of obscured to the user. The second hidden layer is on its turn fully connected two the final output layer, which consists of two nodes. So in this example we feed the model four inputs and we will receive two outputs.

Let’s now focus on a single neuron from our previous example. This neuron still is connected to all inputs, also called features. Using these features, the neuron calculates a single response (or output). A diagram of such a system is depicted in figure 2.

![Logistic unit](https://github.com/dennisbakhuis/Tutorials/raw/master/Logistic_Regression/assets/logisticunit.png)
*Figure 2: single neuron with four input features. The neuron has two operations: a linear part and the activation function.*

The input features are named 𝑓¹, 𝑓², 𝑓³, and 𝑓⁴ and are all connected to the single neuron. This neuron executes two operations. The first operation is a multiplication of the features with a weight vector 𝑊 and adding the bias term 𝑏. The second operation is a so called activation function, indicated here by 𝜎. The output of the neuron is a probability between zero and one. The single neuron acts like a small logistic regression model and therefore, an ANN can be seen as a bunch of interconnected logistic regression models stacked together. While this idea is pretty neat, the underlying truth is a bit more subtle. There are many different architectures for ANNs and they can use various building blocks that act quite different than in this example.

The linear operation in our single neuron is nothing more than a linear regression. Therefore, to understand logistic regression, the first step is to have an idea how linear regression works. The next section will show a step by step example as a recap.

# 2.1 One step back: linear regression
## a) What is a linear regression again?
Linear regression in its simplest form (also called simple linear regression), models a single dependent variable 𝑦 using a single independent variable 𝑥. This may sound daunting, but was this means is that we want to solve the following equation:
```
𝑦 = a𝑥 + 𝑏
```

In the context of machine learning, 𝑥 represents our input data, 𝑦 represents our output data, and by solving we mean to find the best weights (or parameters), represented by 𝑤 and 𝑏 in the linear regression equation. A computer can help us find the best values for 𝑤 and 𝑏, to have the closest match for 𝑦 using the input variable 𝑥.

For the next examples, let us define the following values for x and y:
```
𝑥 = [−2,−1,0,1,2,3,4,5]
𝑦 = [−3,−1,1,3,5,7,9,11]
```

The values for 𝑥 and 𝑦 have a linear relation so we can use linear regression to find (or fit) the best weights and solve the problem. Maybe, by staring long enough at these values, we can discover the relation, however it is much easier to use a computer to find the answer.

If you have stared long enough or just want to know the answer, the relation between 𝑥 and 𝑦 is the following:
```
𝑦 = 2𝑥 + 1
```

In the next section we will use Tensorflow to create our single neuron model and try to ‘solve’ the equation.

## b) An implementation in Tensorflow

Before we start with Tensorflow, we should first organize our input data (𝑥) and output data (𝑦). For this we are going to use Numpy:

In [ ]:
import numpy as np

X = np.array([-2, -1,  0,  1,  2,  3,  4,  5], dtype=np.float)
Y = np.array([-3, -1,  1,  3,  5,  7,  9, 11], dtype=np.float)

In the next step, we will import Tensorflow. It is always a good practice to check which version we are using:

In [ ]:
import tensorflow as tf

tf.__version__

Now we can create a model using Keras, which is now a part of Tensorflow. To do this, we will use the Sequential class, which can stack various layers ‘sequentially’ after each other. We use the Dense class from Keras to create a ‘fully connected’ layer, which consists of a single neuron (unit).

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[1]),
])

The Dense function is used to create layers of many fully connected neurons (logistic units). The parameter units is used to set the amount of neurons. We only use a single unit and therefore, we will set it to one. As this is the first ‘layer’ of our model, we need to tell Tensorflow what shape it can expect as an input. This is only necessary for the first layer.

Now that we have defined the model, we need to use the Compile() method to configure the model for training. The method requires at least two parameters, a loss function and an optimizer. The loss function is a measure for how well the model predicts the actual value. For this example we will use the mean squared error (average of the squared difference between the predicted and the actual value of 𝑦). The ‘learning algorithm’ will try to minimize the loss my adjusting (optimizing) the parameters (weights and bias) for each step. The optimizer defines a method to perform this Optimizing step and a common method is Gradient Descent, or in our case Stochastic Gradient Descent (SGD). This method will become more clear in the next section where we will implement it in plain Numpy.

In [ ]:
model.compile(optimizer='sgd', loss='mean_squared_error')

Next, we will use the Fit() method to let the algorithm learn the best parameters. While Tensorflow has many smart ways to address this problem, what more or less is happing, are the following steps:
1. calculate the prediction $\hat y$ using the current weights of the model
2. calculate the loss of the current values
3. calculate the gradient of the loss function with respect to the parameters ($W$ and $b$)
4. adjust the weights (optimize) using the gradient.
5. repeat for the number of epochs, i.e. the number of times to go through the provided examples (dataset).

If it is not yet clear what this means, it does not really matter yet. It will become more clear in section 2C.

This step will generate a lot of output. Just notice that the loss is indeed decreasing and 'close' the output cell by clicking to the left of this cell.

In [ ]:
model.fit(X, Y, epochs=500)

Now we can use our model to 'predict' values it has never seen. This is sometimes also called inference. Let's try the value of 12. We know that it should be 25. 

In [ ]:
model.predict([12.0])

Why is the value not exactly 25?\
The model calculates the difference between the actual value and the predicted value and creeps slowly towards the actual value. Running the fit procedure longer will get you closer to 25.

This was not that hard, but it might feel dark Jedi force. Therefore, in the next section we will implement this algorithm in plain Python (with the help of Numpy).

## 2C) What's actually happening 'under the hood'
In the previous section we gave a rough overview what Tensorflow is doing under the hood:

1. calculate the prediction $\hat y$ using the current weights of the model
2. calculate the loss of the current values
3. calculate the gradient of the loss function with respect to the parameters ($W$ and $b$)
4. adjust the weights (optimize) using the gradient.
5. repeat for the number of epochs, i.e. the number of times to go through the provided examples (dataset).

We will no code exactly this and hopefully come to a similar result as Tensorflow.

First we define the model parameters. These are the weights $W$, which is just a single value, because we only have a single input. Also we need to define the bias term $b$.

In [ ]:
np.random.seed(2020) # to make reproducible results

In [ ]:
W = 0.01 * np.random.randn(1) # In deep learning it is important to initialize the weights randomly. For our example, 0 would suffice.
b = 0

I called the parameters $W$ and $b$ to match their corresponding official neural network terms: weights and bias. We are however, still calculating the exact same thing as the linear regression problem:
\begin{equation}
y = W x + b
\end{equation}

Next we will define a function that makes a prediction using our current model parameters. In deep learning terms, this is called forward pass. The variable of the predicted value is generally name $\hat y$.

In [ ]:
def forward(X, W, b):
    yhat = W * X + b
    return yhat 

The function is named forward and uses the input vector $X$ and multiplies it with the weight parameter $W$ and adds the bias term $b$. Exactly as we decribed in the equation.

Now we can test the function with the current parameters. Again, we will input a value of 12.0 but of course, it will return gibberish as the weights are randomly initialized.

In [ ]:
yhat = forward(12.0, W, b)
yhat[0]

Indeed not quite right, but we still have to train our model. Before we can do that we need to calculate the current loss. As a loss we used the mean squared error of the predicted value $\hat y$ and the actual value $y$.

\begin{equation}
Loss = \frac{1}{m} \sum_{i=1}^{m} (y - \hat{y})^2
\end{equation}

Hopefully the math does not scare you, but if you take the time, it is not that hard. The variabel $m$ here is the amount of examples (points in the dataset). Our $X$ holds eight values and therefore, $m=8$. When we have a $1/m$ followed by as sum ($\sum$) over $m$ is an average of all the values that are $inside$ the summation. Here, we take the average over all $(y - \hat{y})^2$ which is the difference between the actual value and the predicted value $\hat y$, squared. The square is important because negative difference and a positve differnce would cancel each other out if we would not square the difference. Now that we fully understand the *mean squared error*, we can implement it in code:


In [ ]:
def loss(yhat, Y):
    m = len(yhat)
    loss = 1/m * np.sum(yhat - Y)**2
    return loss

To see what the loss is between our previously calculated value, we can do this:

In [ ]:
loss(yhat, 12)

As we can see, our weights are pretty much off and we need to update them. To do that we need to first calculate the gradients $\delta$Loss/$\delta W$ and $\delta$Loss/$\delta b$. Maybe your differential skills are a bit rusty. The trick is to apply the product rule. We can ignore the sums as these are linear:

\begin{equation}
\frac{\delta}{\delta W} Loss =  \frac{1}{m} \sum_{i=1}^{m}  \frac{\delta}{\delta W} (y - \hat{y})^2 \\
U = (y - \hat{y}) = (y - (Wx + b)) \\
\frac{\delta}{\delta W} Loss = \frac{1}{m} \sum_{i=1}^{m}  \frac{\delta}{\delta U} U^2 \frac{\delta}{\delta W} -(W x + b) \\
\frac{\delta}{\delta W} Loss = \frac{1}{m} \sum_{i=1}^{m}  -2x(y - \hat{y})
\end{equation}

For $\delta$Loss/$\delta b$ we only need to repeat the last step, with respect to $b$:
\begin{equation}
\frac{\delta}{\delta b} Loss = \frac{1}{m} \sum_{i=1}^{m}  \frac{\delta}{\delta U} U^2 \frac{\delta}{\delta b} -(W x + b) \\
\frac{\delta}{\delta b} Loss = \frac{1}{m} \sum_{i=1}^{m}  -2(y - \hat{y})
\end{equation}

We will implement this in the 'backward pass' function. As variable names get a bit long, we will just call them $dW$ and $db$.

In [ ]:
def backward(X, Y, yhat):
    m = len(yhat)
    dW = 1/m * np.sum( -2 * X * (Y - yhat))
    db = 1/m * np.sum( -2 * (Y - yhat))
    return (dW, db)

So now we can get the gradient of our previous test example:

In [ ]:
(dW, db) = backward(12.0, 25, yhat)
(dW, db)

The last function we need, before we can compose our training loop is the update function. This function will 'optimize' our weights one step. This is the actual gradient descent in which we subtract (descent) the gradient from our current weights. Gradient descent is defined as follows:

\begin{equation}
W = W - \alpha \delta W\\
b = b - \alpha \delta b
\end{equation}

Here we have a new parameter $\alpha$ which is called the learning rate. We will set it to 0.01. Our code for the update function is as follows:

In [ ]:
def update(W, b, dW, db, learning_rate = 0.01):
    W = W - learning_rate * dW
    b = b - learning_rate * db
    return (W, b)

To update our curent model paramters we simply do:

In [ ]:
(W, b) = update(W, b, dW, db)
(W, b)

Alright, we have updated our weights for the first time. To improve the weights, we have to repeat this process many times. For this we will write a loop:

In [ ]:
num_iterations = 500

# a reset for W and b
np.random.seed(2020)
W = 0.01 * np.random.randn(1)
b = 0

for i in range(num_iterations):
    yhat = forward(X, W, b)
    l = loss(Y, yhat)
    dW, db = backward(X, Y, yhat)
    W, b = update(W, b, dW, db)
    if i % 100 == 0:
        print(l)

We are in luck and the loss, i.e. the difference between our model prediction and the actual value, is decreasing. How would we now predict, when we input a value of 12.0?

In [ ]:
forward(12.0, W, b)

I hope that the magic box in Tensorflow is now a bit more clear. In the next section we will use our new knowledge for binary logistic regression.

# 3) From Linear Regression to Binary Logistic Regression
## a) What is the difference?
The differences between linear regressions and a logistic regressions are not major. There are two differences from the previous code we created. First, our linear regression model only had a single feature, which we inputted with $x$, meaning that we only had a single weight. In logistic regression, you generally input more than one feature, and each will have its own weight. This will change the previous simple multiplication to a matrix multiplication (dot product). Secondly, we will add a so called *activation function* to map this value between 0 or 1. Let's remind ourselves again of our simple model:
![Logistic unit](https://github.com/dennisbakhuis/Tutorials/raw/master/Logistic_Regression/assets/logisticunit.png)

By convention (from what I have understood) in Tensorflow, the input vector has columns for features, and rows for examples. If we would have 2 datapoints the input matrix would look like this:

\begin{equation}
X = \left( 
\begin{matrix}
f^1_1 & f^2_1 & f^3_1 & f^4_1 \\
f^1_2 & f^2_2 & f^3_2 & f^4_2 
\end{matrix} \right)
\end{equation}

The superscript shows the feature number, the subscript indicates the example.

Each of these inputs are associated with their own weights. The node itself has two explict operations. The first is the dot product between the weights vector and the input vector. The second is the sigmoid funtion. The weight vector $W$ in this example has four weights:

\begin{equation}
X = \left( 
\begin{matrix}
W^1 \\
W^2 \\
W^3 \\
W^4
\end{matrix} \right)
\end{equation}

In the node we first compute the linear part:

\begin{equation}
Z = W X + b
\end{equation}

For our example this will look like this:

\begin{equation}
Z = \left( 
\begin{matrix}
W^1 \\
W^2 \\
W^3 \\
W^4
\end{matrix} \right) \left( 
\begin{matrix}
f^1_1 & f^2_1 & f^3_1 & f^4_1 \\
f^1_2 & f^2_2 & f^3_2 & f^4_2 
\end{matrix} \right) + b = \left(
\begin{matrix}
W^1 f^1_1 + W^2 f^2_1 + W^3 f^3_1 + W^4 f^4_1 + b\\
W^1 f^1_1 + W^2 f^2_1 + W^3 f^3_1 + W^4 f^4_1 + b
\end{matrix}
\right) = \left( 
\begin{matrix}
z_1 \\
z_2 
\end{matrix} \right)
\end{equation}

Notice that the result is only a single value for each example.

For all values of $x$, which can be from $-\infty$ to $+\infty$ (all real numbers), the sigmoid function maps $x$ between 0 and 1. Only values for $x$ close to zero matter, as these are in the 'linear regime'. Very large, or very small are only clipped to 1 and 0 respectively. The mathematical definition of the sigmoid is:

\begin{equation}
A = \frac{1}{1 + e^{-Z}}
\end{equation}

This is all there is to the logistic unit. The sigmoid function gives the node its non-linear character. Many of these units together can do almost magical things. First we will make our first logistic regression model in Tensorflow.

## b) An implementation in Tensorflow
Before we can start we first need some data to do a logistic regression. I downloaded the titantic dataset from Azeem Bootwala from Kaggle to have a play for this example. It can be downloaded from here:\
https://www.kaggle.com/azeembootwala/titanic

In [ ]:
import pandas as pd
df = pd.read_csv('../input/titanic//train_data.csv')
df.shape

First, always inspect the columns and data types:

In [ ]:
df.info()

I expect that Azeem did not save the set using index=False option and therefore we have a 'Unnamed: 0' column. This one is redundant with our current index so we can remove it. Also the PassengerId is not useful for our model, lets remove that column too.

In [ ]:
df = df.drop(['Unnamed: 0', 'PassengerId'], axis=1)

Lets observe some random examples:

In [ ]:
df.sample(5)

Azeem already did some preprocessing. The target variable $Y$ is 'Survived', all other columns are features.\
A short description:
- Sex: 0 or 1 -> male or female
- age: value rescaled between 0 and 1
- fare: ticket price rescaled between 0 and 1
- Pclass_1 .. Pclass_3: One-hot encoded Passenger class
- family size: rescaled value between 0 and 1 of family size.
- title_1 .. title_4: mr, mrs, master, miss one-hot encoded
- emb_1 .. emb_3: Embark location one-hot encoded. 

In total we will have 14 features.

For this example, the data will suffice and I will not go into detail on how this data has become what it is. Honestly, I do not know myself and have just downloaded it ;-).

Lets put these variables in the format we defined before ($X$ and $Y$)

In [ ]:
Y = df['Survived'].to_numpy()
X = df.iloc[:,1:].to_numpy()

Variable $Y$ is the label if they survived and we had 792 examples. So the shape will be $(m, 1)$ where $m$ = 792:

In [ ]:
Y.shape

Numpy omits the 1 for the single column as it is redundant. 

For X, we expect a shape of ($m$, 14):

In [ ]:
X.shape

Now that we have the data, lets create the model in Tensorflow:

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[14], activation='sigmoid'),
])

The model in Tensorflow is very similar to our linear regression model. The input has changed from 1 to 14 features and we added an activation function.

In [ ]:
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])

To configure the model, we again do a compile. For this example I changed the loss into 'binary_cross_entropy'. This is another loss function which works better for binary logistic regression problems. If you are interested in the inner workings I recommend wikipedia. \
I also added the metric 'accuracy' to calculate the accuracy for each epoch. This should improve as we train the model.

In [ ]:
train_history = model.fit(X, Y, epochs=500)

Well, this was all to training a binary logistic classifier in Tensorflow. We achieved an accuracy of ~80% which is not too bad for the effort we put in. 

We can extract the weights $W$ and the bias $b$ from our model. These values we can later compare to our own implementation of the logistic regression:

In [ ]:
W_tf, b_tf = [x.numpy() for x in model.weights]
W_tf, b_tf

We can also plot the loss if we like:

In [ ]:
import matplotlib.pyplot as plt
plt.xkcd()
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.plot(np.arange(500), train_history.history['loss'], 'b-', label='loss')
xlab, ylab = ax.set_xlabel('epoch'), ax.set_ylabel('loss')

Don't mind the nice XKCD wobble :-). It is quite impressive how few steps are required to get to such a result. In the final section we will unveal the Dark Jedi arts being performed by Tensorflow.

## c) What is actually happening?
Well, the steps required are still the same:
1. forward pass
2. calculate loss
3. backward pass
4. update weights
5. repeat

Al steps have minor changes. The forward pass will be a more general dot product and we need to add the activation function. This final activation functions gives a value between 0 and 1, we need to round this to an actual value of 0 or 1. The loss function is binary cross entropy, which is of course different from the mean square error. The backward pass will calculate the gradient of the loss function with respect to $W$ and $b$. As our loss function changed, we will have different differentials. The update weights function is unchanged, the final loop is also very similar.

In Numpy, and in math in general as far as I know, the dot product needs the shapes of the vectors (and matrices) to be compatible:

\begin{equation}
X Y = X_{i,j} Y_{j, k}
\end{equation}

This means that the number of columns of $X$ must be equal to the number of rows in $Y$. To make ourselves a bit easier, we will Transpose our input vector $X$ and flip the vector. This will result that the rows will be features and the columns will be examples.

In [ ]:
X = X.T
X.shape

Lets first get two rows from our dataset to do test calculations:

In [ ]:
Xtry = X[ :, :2]
Ytry = Y[:2]
Xtry.shape

In [ ]:
Ytry

Lets define our weights. As we have 14 features, our vector $W$ will have 14 values. The bias is a constant for the whole node, and is only a single value.

In [ ]:
np.random.seed(2020)
W = 0.01 * np.random.randn(14)
b = 0
W.shape

As we need to calculate the sigmoid function:
\begin{equation}
A = \frac{1}{1 + e^{-Z}}
\end{equation}
lets create it in Python:

In [ ]:
def sigmoid(Z):
    A = 1 / (1 + np.exp(-Z))
    return A

sigmoid(np.array([-100, 0, 0.1, 1000]))

Now lets redefine our forward function to do the dot product and the activation function. The forward pass is defined in two steps:
1. $Z = W X + b$
2. $\sigma(Z)$

Note that $W X$ is a dot product.

In [ ]:
def forward(X, W, b):
    Z = np.dot(W.T, X) + b
    A = sigmoid(Z)
    return A

A = forward(Xtry, W, b)
A

Now that we have actual predictions, we can write our loss function to measure how well our predictions are. This is done with the binary cross entropy, for which I will simply give the equation. 
\begin{equation}
loss = -\frac{1}{m}\sum_{i=1}^{m}y\log(A)+(1-y)\log(1-A)
\end{equation}

It might happen for the first iteration that we try to calculate a $\log(0)$ which is of course not defined. To avoid the warning, we will add a tiny value to our loss. As it is super small, the difference is not noticable, but does help suppress the warning. One less warning a day keeps the ....

In [ ]:
def loss(A, Y, epsilon = 1e-15):
    m = len(A)
    l = -1/m * np.sum( Y * np.log(A + epsilon) + (1 - Y) * np.log(1 - A + epsilon))
    return l

loss(A, Ytry)

Next is the backwards pass. For this, We would need to differentiate the Loss function with $W$ and $b$. Not to  bore you guys, I have provided these functions:
\begin{equation}
 \frac{\partial loss}{\partial W} = \frac{1}{m} \sum_{i=1}^m  X(A - Y)^T \\
 \frac{\partial loss}{\partial b} = \frac{1}{m} \sum_{i=1}^m (A - y)
\end{equation}
In Python, this looks like this:

In [ ]:
def backward(X, Y, A):
    m = len(yhat)
    dW = 1/m * np.dot(X, (A - Y).T)
    db = 1/m * np.sum(A - Y) 
    return (dW, db)

(dW, db) = backward(Xtry, Ytry, A)
dW, db

Almost there, next we need to update the weights.

In [ ]:
def update(W, b, dW, db, learning_rate = 0.01):
    W = W - learning_rate * dW
    b = b - learning_rate * db
    return (W, b)

update(W, b, dW, db)

To compare the results, we can calculate the accuracy. However our activation function returns a probability between 0 and 1. By definition, values <= 0.5 are rounded to 0 and values > 0.5 are rounded to 1. This is slightly different from the round function so we will make our own function for this:

In [ ]:
def roundValue(A):
    return np.uint8( A > 0.5)

yhat = roundValue(A)
yhat

Now we can calculate the accuracy:

In [ ]:
def accuracy(yhat, Y):
    return round(np.sum(yhat==Y) / len(yhat) * 1000) / 10

Now finally lets implement the final loop:

In [ ]:
num_iterations = 8000
lr = 0.01

# Lets just reset W and b
np.random.seed(2020)
W = 0.01 * np.random.randn(14)
b = 0

losses, acces = [], []
for i in range(num_iterations):
    A = forward(X, W, b)
    l = loss(Y, A)
    yhat = roundValue(A)
    acc = accuracy(yhat, Y)
    dW, db = backward(X, Y, A)
    W, b = update(W, b, dW, db, learning_rate=lr)
    losses.append(l)
    acces.append(acc)
    if i % 1000 == 0:
        print('loss:', l, f'\taccuracy: {accuracy(yhat, Y)}%') 

We can plot the loss function of our function:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.plot(np.arange(len(losses)), losses, 'b-', label='loss')
xlab, ylab = ax.set_xlabel('epoch'), ax.set_ylabel('loss')

And the accuracy:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.plot(np.arange(len(acces)), acces, 'b-', label='accuracy')
xlab, ylab = ax.set_xlabel('epoch'), ax.set_ylabel('accuracy')

# 4. Round up
Well, this was it for the tutorial on a Logistic regression. Hopefully you got an idea on how Logistic regression work and that Tensorflow is not only black magic. I found it a great exercise to write these from scratch and as you have seen, it is not that difficult either.

Please let me know if you have any comments or suggestions.